<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install tabula-py
!pip install pdfminer
!pip install PyPDF2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 27 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 3s (1,777 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 65.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.9 MB/s eta 0:00:00
  Created wheel for pdfminer: filename=pdfminer-20191125-py3-none-any.whl size=6140093 sha256=e7cea74df455c9296c157029f56a6c04c5354c2855173d5bcd6b74be7573b4bf
  Stored in directory: /root/.cache/pip/wheels/1c/28/7d/f390b82bb0307deb63ff27a1474fd308ec68ee028cb9ab6283
Successfully built pdfminer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 5.6 MB/s eta 0:00:00


In [58]:
import io
import re
import os
import cv2
import tabula
import PyPDF2
import pytesseract
import pandas as pd
from PIL import Image
from io import StringIO
from google.colab import files
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [59]:
file_objects = files.upload()

Saving Posicao Consolidada XP - Cliente 04.pdf to Posicao Consolidada XP - Cliente 04 (2).pdf


In [60]:
# Converte o conteúdo do PDF para texto
def convert_pdf_to_txt(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text

#def remove_header_and_footer(text, cm, tm, fontDict, fontSize):
#    y = tm[5]
#    if y > 50 and y < 720:
#        parts.append(text)

def get_content_of_tables(pdf_file):
    # Extrair todas as tabelas do arquivo PDF
    tables = tabula.read_pdf(pdf_file, multiple_tables=True, pages="all")

    return tables


def get_text_from_page_images(reader):

    # Inicializa variáveis
    texts = []
    page_number = 0

    # Percore todas as páginas do documento
    for page in reader.pages:
        
        page_number += 1

        try:
            # Obtém os recursos da página
            xObject = page['/Resources']['/XObject'].get_object()

            image_number = 0
            # Percorre todos os objetos X da página
            for obj in xObject:

                image_number += 1

                # Verifica se o objeto é uma imagem
                if xObject[obj]['/Subtype'] == '/Image':
                    size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                    data = xObject[obj].get_data()
                    if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                        mode = "RGB"
                    else:
                        mode = "P"

                    # Cria uma imagem a partir dos dados
                    img = Image.frombytes(mode, size, data)
                    image_name = file_name[0:25] + "_" + str(page_number) + "_" + str(image_number) + ".png"
                    #image_path = pdf_path = "/content/gdrive/My Drive/Temp/" + image_name

                    # Salva a imagem
                    img.save(image_name)

                    # Carrega a imagem
                    image = cv2.imread(image_name)

                    # Converte a imagem para escala de cinza
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                    # Executa o OCR usando o Tesseract
                    text = pytesseract.image_to_string(image).strip()

                    # Adiciona o texto obtido ao array de textos
                    texts.append(text)
        except:
            texts.append(f'Não foi possível ler as imagens da página {page_number}')
    
    return texts

def create_new_df():

    # Cria um DataFrame utilizado no consolidado final
    return pd.DataFrame({
        'CLASSIFICAÇÃO': pd.Series(dtype='str'),
        'ATIVO': pd.Series(dtype='str'),
        'EXPOSIÇÃO': pd.Series(dtype='str'),
        'INSTITUIÇÃO': pd.Series(dtype='str'),
        'ATUAL': pd.Series(dtype='str'),
        'DATA': pd.Series(dtype='str'),
        'ON/OFF': pd.Series(dtype='str'),
        'DATA DE VENCIMENTO': pd.Series(dtype='str')
    })

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}(3|4|5|6|11|32|33|34|35)\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações", "FI Açoes"
        ],
        "Renda Fixa": [
            "Renda Fixa","RF","CDB","LC","LCA","LCI","CRA","CRI","Deb","Debenture",
            "Debênture","Tesouro","Pré","Pós","Pre","Pos","IPCA","IPCA+","LTN","NTN",
            "NTN-B","NTNB","DI","REF DI","CP","Credito Privado","Crédito Privado",
            "LP","Longo Prazo","Previdencia","Previdência","Prev","VGBL","PGBL"
        ],
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","COE","Multiestratégia",
            "Multiestratégia"
        ]
    }

    classes = []
    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo, flags=(re.IGNORECASE))
            if founded and not classificacao in classes:
                classes.append(classificacao)

    if len(classes) > 0:
        return ' / '.join(classes)

    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"

    return "***"

def get_pdf_texts():

    text_pypdf = ""
    text_pdfminer = ""

    for file in file_objects.keys():

        file_name, file_extension = os.path.splitext(file)

        if file_extension != '.pdf':
            continue

        # Lê o conteúdo do arquivo PDF
        text_pdfminer = convert_pdf_to_txt(file)
        
        reader = PyPDF2.PdfReader(file)
        page_number = 0

        for page in reader.pages:

            page_number += 1
            text_pypdf = text_pypdf + page.extract_text() #visitor_text=remove_header_and_footer)
        
        # print(get_text_from_page_images(reader))
    
    return text_pypdf, text_pdfminer

In [61]:
# Cria o DataFrame que conterá o consolidado final
df_result = create_new_df()

# Obtém os textos do pdf
text_pypdf, text_pdfminer = get_pdf_texts()

In [67]:
print(text_pypdf)

DISTRIBUIÇÃO DA CARTEIRA ATUAL
CLASSE DE PRODUTOS 
 60.3%60.3%
Ações
R$ 3.729.825,00  35.7%35.7%
Renda Fixa
R$ 2.207.124,14
 3.6%3.6%
Fundos de Investimentos
R$ 222.638,62  0.35%0.35%
Opções
R$ -21.600,00
 0.05%0.05%
Proventos
R$ 2.903,14VALORES DA CARTEIRA 
PATRIMÔNIO
R$ 6.140.918,87
INVESTIMENTO
R$ 6.140.890,90
SALDO DISPONÍVEL EM CONTA
R$ 27,97
NÍVEL DE ADERÊNCIA DA CARTEIRA 
BAIXO MÉDIO ALTO
 
PREFIXADAPREFIXADA
ATIVO APLICAÇÃO CARÊNCIA VENCIMENTO TAXA DISPONÍVEL GARANTIAVALOR 
APLICADOPOSIÇÃOVALOR 
LÍQUIDO
LFSN BANCO SAFRA S/A - JUL/2025 07/02/2019 24/07/2025 24/07/2025 + 9,77% 1 0 R$ 383.638,76 R$ 543.367,46 R$ 519.408,16
CDB AGIBANK - MAI/2025 18/05/2022 17/05/2025 19/05/2025 + 14,25% 100 0 R$ 100.000,00 R$ 106.492,98 R$ 105.032,06
CDB BANCO XP S.A. - JUL/2023 13/07/2022 13/07/2023 13/07/2023 + 14,80% 125 0 R$ 125.000,00 R$ 130.598,80 R$ 129.339,08
CDB BMG - JUL/2024 20/07/2022 19/07/2024 19/07/2024 + 15,60% 28378 0 R$ 28.378,00 R$ 29.629,16 R$ 29.347,65
CDB BMG - SET/2024 20/09

In [69]:
# Regex para obter a data de emissão
regex_0 = "(?<=Data da Consulta: )\d{2}\/\d{2}\/\d{4}"
data_emissao = re.search(regex_0, text_pypdf, flags=(re.IGNORECASE)).group(0)

# Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
regex_1 = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\b \d+ \d+ \d+ \d+ \d+ \d+ \d+ R\$ [\d,.]+[.,]\d{2} (R\$ [\d,.]+[.,]\d{2})'
result_1 = re.findall(regex_1, text_pypdf, flags=(re.IGNORECASE | re.DOTALL | re.MULTILINE))

#regex_2 = r'\n(.+?) - [A-Z]{3}\/\d{4} \d{2}\/\d{2}\/\d{4} \d{2}\/\d{2}\/\d{4} (\d{2}\/\d{2}\/\d{4}) [\+\-] [\d,.]+% \d+ \d+ R\$ [\d,.]+[.,]\d{2} (R\$ [\d,.]+[.,]\d{2}) R\$ [\d,.]+[.,]\d{2}'
#regex_2 = r'(?<=\n)(.+?)(\n| )*-(\n| )*[A-Z]{3}(\n|.)*\/\d{4}(\n|.)*\d{2}\/\d{2}\/\d{4} \d{2}\/\d{2}\/\d{4} \d{2}\/\d{2}\/\d{4}(\n| )*[A-Z\-]*(\n| )[\+\-]( |\n)*[\d,.]+%(\n| )*\d+(\n| )*\d+(\n| )*R\$(\n| )*[\d,.]+[.,]\d{2}(\n| )*(R\$(\n| )*[\d,.]+[.,]\d{2})(\n| )*R\$(\n| )*[\d,.]+[.,]\d{2}'
regex_2 = r'(?<=\n)(.+?)[\n ]*-[\n ]*[A-Z]{3}\/\d{4}[\n ]*\d{2}\/\d{2}\/\d{4}[\n ]*\d{2}\/\d{2}\/\d{4}[\n ]*(\d{2}\/\d{2}\/\d{4})[\n ]*[A-Z\-]*[\n ]*[\+\-][\n ]*[\d,.]+%[\n ]*\d+[\n ]*\d+[\n ]*R\$[\n ]*[\d,.]+[.,]\d{2}[\n ]*(R\$[\n ]*[\d,.]+[.,]\d{2})[\n ]*R\$[\n ]*[\d,.]+[.,]\d{2}'
result_2 = re.findall(regex_2, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))

# Preenche o consolidado final
for res in result_1:

    df_result = df_result.append({
        'CLASSIFICAÇÃO': obtem_classificacao(res[0]),
        'ATIVO': res[0],
        'EXPOSIÇÃO': "Real",
        'INSTITUIÇÃO': "XP Investimentos",
        'ATUAL': res[2],
        'DATA': data_emissao,
        'ON/OFF': "ON",
        'DATA DE VENCIMENTO': "-"
    }, ignore_index=True)

# Preenche o consolidado final
for res in result_2:

    print(res)

#df_result

('LFSN BANCO SAFRA S/A', '24/07/2025', 'R$ 543.367,46')
('CDB AGIBANK', '19/05/2025', 'R$ 106.492,98')
('CDB BANCO XP S.A.', '13/07/2023', 'R$ 130.598,80')
('CDB BMG', '19/07/2024', 'R$ 29.629,16')
('CDB BMG', '19/09/2024', 'R$ 53.886,21')
('LF BANCO ALFA', '24/08/2023', 'R$ 56.141,60')
('LF BANCO ALFA', '24/08/2023', 'R$ 112.283,19')
('CDB PAGBANK', '02/05/2023', 'R$ 8.543,47')
('CDB ORIGINAL', '29/06/2026', 'R$ 69.260,60')
('CDB ORIGINAL', '29/06/2026', 'R$ 106.199,58')
('CDB ORIGINAL', '28/07/2023', 'R$ 145.921,80')
('CDB BMG', '30/09/2024', 'R$ 26.756,18')
('CDB NBC BANK', '14/10/2024', 'R$ 44.360,15')
('CDB BANCO C6 S.A.', '26/11/2025', 'R$\n200.829,14')
('CDB BANCO PAN S/A', '09/03/2027', 'R$ 68.183,25')
('DEB UNIDAS', '15/09/2031', 'R$ 50.297,21')
('CDB DAYCOVAL', '28/10/2023', 'R$\n226.704,78')
('CDB BANCO VOITER S.A.', '29/10/2023', 'R$ 56.742,27')
('CDB AGIBANK', '07/04/2025', 'R$ 15.984,62')
('CDB BANCO C6 CONSIGNADO S.A.', '11/10/2023', 'R$ 57.251,69')
('CDB BANCO XP S.A.',

In [28]:
# Regex para obter a data de emissão
regex_0 = "(?<=Movimenta..o de \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
data_emissao = re.search(regex_0, text_pdfminer, flags=(re.IGNORECASE)).group(0)

# Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
regex_1 = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
result_1 = re.search(regex_1, text_pdfminer, flags=(re.IGNORECASE | re.DOTALL)).group(0)

# Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
regex_2 = "^.*[a-zA-Z] .*$"
result_2 = re.finditer(regex_2, result_1, flags=(re.MULTILINE))

# Gera um dicionário de termos com espaço
dictionary = {}
print("TEXTOS A SEREM CORRIGIDOS:")
for result in result_2:
    value = result.group(0)
    dictionary[value.replace(" ","")] = value
    print(" - " + value)

TEXTOS A SEREM CORRIGIDOS:
 - Valor Aplicado
 - Valor Bruto
 - SulAmérica Inflatie FI Renda Fixa Longo Prazo
 - XP Referenciado FI Referenciado DI CP
 - XPA MM II FIC FIM
 - XPA RV FIC FIM
 - XPA Vértice Crédito FIM CP


In [29]:
# Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
regex_3 = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
result_3 = re.search(regex_3, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

# Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
result_3 = result_3.replace(" ",";")
print(result_3)

Fundo;Cota;Quantidade;ValorAplicado;ValorBruto;IR;IOF;ValorLiquido
SulAméricaInflatieFIRendaFixaLongoPrazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XPReferenciadoFIReferenciadoDICP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPAMMIIFICFIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPARVFICFIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPAVérticeCréditoFIMCP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [30]:
# Altera todos os textos incorretos para a versão correta
for key in dictionary:
    if key in result_3:
        result_3 = result_3.replace(key,dictionary[key])

print(result_3)

Fundo;Cota;Quantidade;Valor Aplicado;Valor Bruto;IR;IOF;ValorLiquido
SulAmérica Inflatie FI Renda Fixa Longo Prazo;49,03907260;485,46020696;21.434,59;23.806,52;13,80;0,00;23.792,72
XP Referenciado FI Referenciado DI CP;2,55168782;9.607,86926679;23.303,62;24.516,28;272,84;0,00;24.243,44
XPA MM II FIC FIM;1,48495549;206.015,91600648;231.165,15;305.924,47;2.095,63;0,00;303.828,84
XPA RV FIC FIM;1,51471586;86.244,99799744;119.600,00;130.636,67;1.655,50;0,00;128.981,17
XPA Vértice Crédito FIM CP;1,31704969;62.558,82325043;67.664,32;82.393,08;816,74;0,00;81.576,34



In [31]:
# Converte o CSV para o tipo DataFrame
df = pd.read_csv(StringIO(result_3), sep=";")
df

,Fundo,Cota,Quantidade,Valor Aplicado,Valor Bruto,IR,IOF,ValorLiquido
0,SulAmérica Inflatie FI Renda Fixa Longo Prazo,"49,03907260","485,46020696","21.434,59","23.806,52","13,80","0,00","23.792,72"
1,XP Referenciado FI Referenciado DI CP,"2,55168782","9.607,86926679","23.303,62","24.516,28","272,84","0,00","24.243,44"
2,XPA MM II FIC FIM,"1,48495549","206.015,91600648","231.165,15","305.924,47","2.095,63","0,00","303.828,84"
3,XPA RV FIC FIM,"1,51471586","86.244,99799744","119.600,00","130.636,67","1.655,50","0,00","128.981,17"
4,XPA Vértice Crédito FIM CP,"1,31704969","62.558,82325043","67.664,32","82.393,08","816,74","0,00","81.576,34"


In [32]:
# Preenche o consolidado final
for index, row in df.iterrows():

    df_result = df_result.append({
        'CLASSIFICAÇÃO': obtem_classificacao(row['Fundo']),
        'ATIVO': row['Fundo'],
        'EXPOSIÇÃO': "Real",
        'INSTITUIÇÃO': "XP Investimentos",
        'ATUAL': row["Valor Bruto"],
        'DATA': data_emissao,
        'ON/OFF': "ON",
        'DATA DE VENCIMENTO': "-"
    }, ignore_index=True)

df_result

,CLASSIFICAÇÃO,ATIVO,EXPOSIÇÃO,INSTITUIÇÃO,ATUAL,DATA,ON/OFF,DATA DE VENCIMENTO
0,Renda Fixa,SulAmérica Inflatie FI Renda Fixa Longo Prazo,Real,XP Investimentos,"23.806,52",11/11/2022,ON,-
1,Renda Fixa,XP Referenciado FI Referenciado DI CP,Real,XP Investimentos,"24.516,28",11/11/2022,ON,-
2,Multimercado,XPA MM II FIC FIM,Real,XP Investimentos,"305.924,47",11/11/2022,ON,-
3,Multimercado,XPA RV FIC FIM,Real,XP Investimentos,"130.636,67",11/11/2022,ON,-
4,Renda Fixa / Multimercado,XPA Vértice Crédito FIM CP,Real,XP Investimentos,"82.393,08",11/11/2022,ON,-


In [33]:
print("Disponibilizando arquivo de consolidado...")

for file in file_objects.keys():

    file_name, file_extension = os.path.splitext(file)

    df_result.to_excel(file_name + ".xlsx", index=False) 
    files.download(file_name + ".xlsx")

Disponibilizando arquivo de consolidado...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>